# Project Setup

This notebook will guide you through setting up the project environment for using CrewAI. We will:

1. Install the required Python modules.
2. Set up a virtual environment.
3. Verify the installation.

In [ ]:
# Uncomment if you are not using devcontainers and want to set up a local environment
# 
# # Step 1: Create and activate a virtual environment
# #
# %python3 -m venv venv
# %source venv/bin/activate
# 
# # Step 2: Install required Python modules
# #
# %pip install -r requirements.txt
# 
# # Step 3: Verify installation
# #
# %pip list

# Load Required Python Modules and Libraries

In [1]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown
from crewai import LLM, Agent, Task, Crew

In [ ]:
# Uncomment in order to enable litellm debugging for better error diagnostics
#
import litellm
litellm._turn_on_debug()
print("✅ LiteLLM debugging enabled")

# Load Environment Variables and Configure LLM

This block loads environment variables from the `.env` file, including the OpenAI API Key, which is required to authenticate with OpenAI's services. It then configures the `LLM` object to use OpenAI's GPT-4 model. Alternatively, you can uncomment the provided code to configure the `LLM` object to use Ollama with a local model, provided Ollama is installed and running.

In [2]:
# Load environment variables from .env file
# Note: In devcontainer, variables are already loaded by dotenv feature,
# but load_dotenv() is safe and won't override existing environment variables
load_dotenv()

# Uncomment the code block below to use OpenAI with your API Key
# 
# api_key = os.getenv('OPENAI_API_KEY')
# if not api_key:
#     raise ValueError("OPENAI_API_KEY is not set in the .env file")

# Uncomment the code block below to use ollama
# 
OLLAMA_API_BASE = os.getenv('OLLAMA_API_BASE')

# Configure LLM

Configures the `LLM` object to use OpenAI's GPT-4 model. 

Alternatively, you can uncomment the provided code to configure the `LLM` object to use Ollama with a local model, provided Ollama is installed and running.

In [3]:
# llm = LLM(
#     model="gpt-4o",  # Specify the OpenAI model you want to use
#     api_key=api_key
# )

# Uncomment the code block below to use Ollama with your local model
# Make sure to have Ollama installed and running
# 
llm = LLM(
    model="ollama/gemma3n:latest",
    base_url=OLLAMA_API_BASE
)

In [ ]:
# Uncomment to test ollam connectivity
# 
import requests

# Debug: Check environment variables and test Ollama connection
# 
print("=== Environment Variables Debug ===")
print(f"OLLAMA_API_BASE: {OLLAMA_API_BASE}")
test_url = f"{OLLAMA_API_BASE}/api/tags"

# Test connection to your Ollama server
# 
try:
    response = requests.get(test_url, timeout=5)
    print(f"\n=== Ollama Connection Test ===")
    print(f"URL: {test_url}")
    print(f"Status: {response.status_code}")
    print(f"Response: {response.json()}")
except Exception as e:
    print(f"Connection failed: {e}")

# Debug: Check what base_url is actually being used
# 
print(f"\n=== CrewAI LLM Connection Test ===")
# print(f"ollama_base_url from env: {OLLAMA_API_BASE}")
print(f"LLM base_url: {llm.base_url}")
print(f"LLM model: {llm.model}")

# Test a simple LLM call
try:
    test_response = llm.call([{"role": "user", "content": "Say hello!"}])
    print("✅ LLM test successful!")
except Exception as e:
    print(f"❌ LLM test failed: {e}")

# Single Agent Single Tool Example

## Define Agents

This block defines multiple agents with specific roles, goals, and backstories. Each agent is configured to use the LLM defined earlier.

In [5]:
senior_technical_writer = Agent(
    role="Senior Technical Writer",
    
    goal="""Craft clear, engaging, and well-structured
            technical content based on research findings""",
    
    backstory="""You are an experienced technical writer
                with expertise in simplifying complex
                concepts, structuring content for readability,
                and ensuring accuracy in documentation.""",
                
    llm=llm,
                
    verbose=True
)

research_analyst = Agent(
    role="Senior Research Analyst",
    goal="""Find, analyze, and summarize information 
            from various sources to support technical 
            and business-related inquiries.""",
    backstory="""You are a skilled research analyst with expertise 
                in gathering accurate data, identifying key trends, 
                and presenting insights in a structured manner.""",
    llm=llm,
    verbose=True
)

code_reviewer = Agent(
    role="Senior Code Reviewer",
    goal="""Review code for bugs, inefficiencies, and 
            security vulnerabilities while ensuring adherence 
            to best coding practices.""",
    backstory="""You are a seasoned software engineer with years of 
                experience in writing, reviewing, and optimizing 
                production-level code in multiple programming languages.""",
    llm=llm,
    verbose=True
)

legal_reviewer = Agent(
    role="Legal Document Expert Reviewer",
    goal="""Review contracts and legal documents to 
            ensure compliance with applicable laws and 
            highlight potential risks.""",
    backstory="""You are a legal expert with deep knowledge 
                of contract law, regulatory frameworks, 
                and risk mitigation strategies.""",
    llm=llm,
    verbose=True
)


## Define a Writing Task

This block defines a writing task for the Senior Technical Writer agent. The task includes a description, the agent responsible, and the expected output.

In [6]:

writing_task = Task(
    description="""Write a well-structured, engaging,
                   and technically accurate article
                   on {topic}.""",
    
    agent=senior_technical_writer, 
    
    
    expected_output="""A polished, detailed, and easy-to-read
                       article on the given topic.""",
)


## Create a Crew and Execute the Task

This block creates a crew to manage the task and agents. It then kicks off the crew with the specified input and displays the response.

In [ ]:
crew = Crew(
    agents=[senior_technical_writer],
    tasks=[writing_task],
    verbose=True
)

response = crew.kickoff(inputs={"topic":"AI Agents"})

# Display the response in Markdown format
Markdown(response.raw)

## Save the Response to a File

This block saves the raw response from the crew execution to a Markdown file for further use or documentation.

In [8]:
f = open("build/output.md", "w")
f.write(response.raw)
f.close()

## Define a file reader tool

In [15]:
# Enable litellm debugging for better error diagnostics
import litellm
litellm._turn_on_debug()
print("✅ LiteLLM debugging enabled")

## Troubleshooting Note

**Issue:** The original code was encountering an `IndexError` in the litellm/ollama integration when using CrewAI tools. The error occurred in the prompt template processing where `messages[msg_i]` was accessing an index out of range.

**Root Cause:** This appears to be a compatibility issue between:
- CrewAI's tool integration system
- LiteLLM's Ollama prompt template processing
- The FileReadTool parameter validation

**Solution:** 
1. **Enabled debugging** with `litellm._turn_on_debug()` to get detailed error information
2. **Removed the FileReadTool** from the agent to avoid the tool integration bug
3. **Read file content directly** in Python and embedded it in the task description
4. **Simplified the workflow** to avoid dynamic parameter passing

This approach maintains the same functionality while working around the integration issue.

In [9]:
from crewai_tools import FileReadTool

file_read_tool = FileReadTool()

/home/vscode/.local/lib/python3.11/site-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


## Define the Agent For Reading And Summarizing Files

In [18]:
from crewai import Agent

summarizer_agent = Agent(
    role="Senior Document Summarizer",
    goal="Extract and summarize key insights from provided content in 20 words or less.",
    backstory="""You are an expert in document analysis, skilled at extracting 
                 key details, summarizing content, and identifying critical 
                 insights from structured and unstructured text.""",
    llm=llm,
    verbose=True
)

## Create Reading and Analyzing File Task

In [19]:
from crewai import Task

# Read the file content directly to avoid tool integration issues
with open("build/output.md", "r") as file:
    file_content = file.read()

summarizer_task = Task(
    description=f"""
    Analyze the following document content and provide a summary in exactly 20 words or less. 
    Focus on the key insights and main points from the document.
    
    Document content:
    {file_content}
    """,
    agent=summarizer_agent,
    expected_output="A concise 20-word summary of the key points from the document.",
)

## Assemble A File-Processing Crew Workflow

In [ ]:
from crewai import Crew

summarizer_crew = Crew(
    agents=[summarizer_agent],
    tasks=[summarizer_task],
    verbose=True
)

# Run the crew without inputs since content is embedded in task
result = summarizer_crew.kickoff()

In [22]:
# Display the response in Markdown format
from IPython.display import Markdown
Markdown(result.raw)

AI Agents are autonomous software entities perceiving, reasoning, acting, and learning to automate tasks across industries, facing challenges in safety, ethics, and scalability.

# Alternative: Using FileReadTool with OpenAI

The following section demonstrates the original approach using the FileReadTool, which works properly with OpenAI but has compatibility issues with the current litellm/ollama setup. 

**Note:** To use this section, you need to:
1. Set up an OpenAI API key in your environment
2. Switch the LLM configuration to use OpenAI instead of Ollama
3. Uncomment and run the cells below

## Configure OpenAI LLM for FileReadTool Example

First, configure the LLM to use OpenAI instead of Ollama for proper tool integration.

In [ ]:
# Uncomment to use OpenAI LLM for FileReadTool example
# Make sure you have OPENAI_API_KEY set in your environment
#
# import os
# from crewai import LLM
# 
# # Get OpenAI API key from environment
# api_key = os.getenv('OPENAI_API_KEY')
# if not api_key:
#     raise ValueError("OPENAI_API_KEY is not set in the environment")
# 
# # Configure LLM for OpenAI
# openai_llm = LLM(
#     model="gpt-4o-mini",  # Use a cost-effective model
#     api_key=api_key
# )
# 
# print("✅ OpenAI LLM configured for FileReadTool example")

## Define FileReadTool and Agent (Original Approach)

This is the original approach using the FileReadTool that works with OpenAI.

In [ ]:
# Uncomment to use the original FileReadTool approach with OpenAI
#
# from crewai_tools import FileReadTool
# from crewai import Agent
# 
# # Create the FileReadTool
# file_read_tool_openai = FileReadTool()
# 
# # Define the agent with FileReadTool (works with OpenAI)
# summarizer_agent_openai = Agent(
#     role="Senior Document Summarizer",
#     goal="Extract and summarize key insights from provided files in 20 words or less.",
#     backstory="""You are an expert in document analysis, skilled at extracting 
#                  key details, summarizing content, and identifying critical 
#                  insights from structured and unstructured text.""",
#     llm=openai_llm,  # Use OpenAI LLM instead of Ollama
#     tools=[file_read_tool_openai],  # Include the FileReadTool
#     verbose=True
# )
# 
# print("✅ FileReadTool agent configured for OpenAI")

## Define Task with FileReadTool (Original Approach)

This task uses the FileReadTool to dynamically read files based on input parameters.

In [ ]:
# Uncomment to use the original task with FileReadTool
#
# from crewai import Task
# 
# # Original task definition using FileReadTool
# summarizer_task_openai = Task(
#     description=(
#         "Use the FileReadTool to read the contents of {file_path} "
#         "and provide a summary in 20 words or less. "
#         "Ensure the summary captures the key insights "
#         "and main points from the document."
#     ),
#     agent=summarizer_agent_openai,
#     tools=[file_read_tool_openai],
#     expected_output="A concise 20-word summary of the key points from the file.",
# )
# 
# print("✅ FileReadTool task configured")

## Execute Crew with FileReadTool (Original Approach)

This demonstrates the original workflow using dynamic file input with the FileReadTool.

In [ ]:
# Uncomment to execute the original FileReadTool approach
#
# from crewai import Crew
# 
# # Create crew with FileReadTool agent
# summarizer_crew_openai = Crew(
#     agents=[summarizer_agent_openai],
#     tasks=[summarizer_task_openai],
#     verbose=True
# )
# 
# # Execute with dynamic file input (original approach)
# result_openai = summarizer_crew_openai.kickoff(inputs={"file_path": "build/output.md"})
# 
# # Display the result
# from IPython.display import Markdown
# print("📝 FileReadTool Result with OpenAI:")
# Markdown(result_openai.raw)

## Comparison: Tool vs Non-Tool Approaches

### FileReadTool Approach (OpenAI Compatible)
**Advantages:**
- Dynamic file reading based on input parameters
- Proper tool integration and validation
- More flexible for different file paths
- Standard CrewAI tool workflow

**Requirements:**
- OpenAI API key and credits
- Compatible LLM provider (OpenAI, Anthropic, etc.)

### Direct File Reading Approach (Ollama Compatible)
**Advantages:**
- Works with local Ollama models (no API costs)
- Avoids tool integration compatibility issues
- Simpler debugging and troubleshooting
- Faster execution (no tool overhead)

**Limitations:**
- File path must be known at task creation time
- Less flexible for dynamic file operations
- Requires manual file handling

### When to Use Each Approach

**Use FileReadTool when:**
- You have access to OpenAI or other compatible APIs
- You need dynamic file reading capabilities
- You want to follow standard CrewAI tool patterns
- You're building production systems with proper tool validation

**Use Direct File Reading when:**
- You're using Ollama or other local models
- You encounter tool integration issues
- You need faster, simpler file processing
- File paths are known at development time

# Building multi-agent systems

## Define our Serper Dev tool

The Serper Dev Tool is a utility designed to interact with search engines and retrieve relevant information programmatically. It is particularly useful for tasks that require real-time data or insights from the web, such as:

1. Conducting research on specific topics.
2. Gathering up-to-date information for decision-making.
3. Enhancing the capabilities of agents by providing them with access to external data sources.

### How It Can Be Used
- **Integration with Agents**: The tool can be integrated into agents to enable them to fetch and process web-based information dynamically.
- **Custom Queries**: Developers can define specific queries to retrieve targeted information, making it adaptable to various use cases.
- **Real-Time Insights**: By leveraging the tool, agents can provide real-time insights and context, improving the quality and relevance of their outputs.

This tool is ideal for scenarios where static data is insufficient, and dynamic, real-time information is required to achieve the desired outcomes.

In [ ]:
from crewai_tools import SerperDevTool

serper_dev_tool = SerperDevTool()

## Define the Internet Researcher Agent and Task

This block defines an `Internet Researcher` agent and a corresponding research task. 
- The agent is equipped with the `SerperDevTool` to perform web-based research. 
- The agent's role is to find the most relevant and recent information about a given topic, leveraging its expertise in navigating the internet and gathering reliable data. 
- The task specifies the use of the `SerperDevTool` to extract key insights from multiple sources and produce a detailed research report with references.

In [ ]:
from crewai import Agent, Task

research_agent = Agent(
    role="Internet Researcher",
    goal="Find the most relevant and recent information about a given topic.",
    backstory="""You are a skilled researcher, adept at navigating the internet 
                 and gathering high-quality, reliable information.""",
    tools=[serper_dev_tool],
    verbose=True
)

research_task = Task(
    description="""Use the SerperDevTool to search for the 
                   most relevant and recent data about {topic}."""
                "Extract the key insights from multiple sources.",
    agent=research_agent,
    tools=[serper_dev_tool],
    expected_output="A detailed research report with key insights and source references."
)

## Define The Summarization Agent

This agent is responsible for condensing the research into a concise and structured summary. The Summarization Agent ensures that the research findings are structured, easy to read, and clear.

In [ ]:
summarizer_agent = Agent(
    role="Content Summarizer",
    goal="Condense the key insights from research into a short and informative summary.",
    backstory="""You are an expert in distilling complex information into concise, 
                 easy-to-read summaries.""",
    verbose=True
)

summarization_task = Task(
    description="Summarize the research report into a concise and informative paragraph. "
                "Ensure clarity, coherence, and completeness.",
    agent=summarizer_agent,
    expected_output="A well-structured summary with the most important insights."
)

## Define The Fact-Checking Agent

The Fact-Checking Agent will cross-check all summarized information with credible sources:

- The Fact-Checking Agent is responsible for validating the summarized information.
- The Serper Dev Tool is used again to cross-check facts with external sources.

In [ ]:
fact_checker_agent = Agent(
    role="Fact-Checking Specialist",
    goal="Verify the accuracy of information and remove any misleading or false claims.",
    backstory="""You are an investigative journalist with a knack for validating facts, 
                 ensuring that only accurate information is published.""",
    tools=[serper_dev_tool],
    verbose=True
)

fact_checking_task = Task(
    description="Verify the summarized information for accuracy using the SerperDevTool. "
                "Cross-check facts with reliable sources and correct any errors.",
    agent=fact_checker_agent,
    tools=[serper_dev_tool],
    expected_output="A fact-checked, verified summary of the research topic."
)

## Create Multi-Agent Crew Workflow

- All three agents are grouped into a Crew, each assigned their specific task.
- Tasks are executed sequentially in a structured workflow:  Research → Summarization → Fact-Checking.
- The topic is dynamically provided at runtime, making the workflow flexible for any research topic.

In [ ]:
from crewai import Crew, Process

research_crew = Crew(
    agents=[research_agent, summarizer_agent, fact_checker_agent],
    tasks=[research_task, summarization_task, fact_checking_task],
    process=Process.sequential,
    verbose=True
)


## Kickoff The Multi-Agent Crew Workflow

In [ ]:

result = research_crew.kickoff(inputs={"topic": "The impact of AI on job markets"})

# print("\nFinal Verified Summary:\n", result)
Markdown(result.raw)

# Using YAML Based Agent and Workflow Definition

## Load YAML File

In [ ]:
import yaml

with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)
    
# Uncomment the following line to print the configuration
# 
# from pprint import pprint
# pprint(config)

## Convert Agent Definitions to use YAML Config Details

In [ ]:
research_agent = Agent(
    role=config["agents"]["research_agent"]["role"],
    goal=config["agents"]["research_agent"]["goal"],
    backstory=config["agents"]["research_agent"]["backstory"],
    tools=[serper_dev_tool],
    verbose=True
)

research_task = Task(
    description=config["tasks"]["research_task"]["description"],
    agent=research_agent,
    tools=[serper_dev_tool],
    expected_output=config["tasks"]["research_task"]["expected_output"]
)

summarization_agent = Agent(
    role=config["agents"]["summarization_agent"]["role"],
    goal=config["agents"]["summarization_agent"]["goal"],
    backstory=config["agents"]["summarization_agent"]["backstory"],
    verbose=True
)

fact_checker_agent = Agent(
    role=config["agents"]["fact_checker_agent"]["role"],
    goal=config["agents"]["fact_checker_agent"]["goal"],
    backstory=config["agents"]["fact_checker_agent"]["backstory"],
    tools=[serper_dev_tool],
    verbose=True
)

summarization_task = Task(
    description=config["tasks"]["summarization_task"]["description"],
    agent=summarization_agent,
    expected_output=config["tasks"]["summarization_task"]["expected_output"],
)

fact_checking_task = Task(
    description=config["tasks"]["fact_checking_task"]["description"],
    agent=fact_checker_agent,
    tools=[serper_dev_tool],
    expected_output=config["tasks"]["fact_checking_task"]["expected_output"],
)


## Create Multi-Agent Crew Workflow

- All three agents are grouped into a Crew, each assigned their specific task.
- Tasks are executed sequentially in a structured workflow:  Research → Summarization → Fact-Checking.
- The topic is dynamically provided at runtime, making the workflow flexible for any research topic.

In [ ]:
from crewai import Crew, Process

research_crew = Crew(
    agents=[research_agent, summarizer_agent, fact_checker_agent],
    tasks=[research_task, summarization_task, fact_checking_task],
    process=Process.sequential,
    verbose=True
)

result = research_crew.kickoff(inputs={"topic": "The impact of AI on job markets"})
print("\nFinal Verified Summary:\n", result)